# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [38]:
import numpy as np
import pandas as pd

In [39]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [40]:
config = pd.read_csv('config.csv').iloc[0]
print(config)

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object


In [41]:
user = config['id']
k_value = config['k']

### Read the Data

In [42]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [43]:
user_likes = dict()

In [44]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [45]:
user_index={}
index_user={}
x=0
for k,v in user_likes.items():
    user_index[k]=x
    index_user[x]=k
    x+=1

image_index={}
x=0
for k,v in user_likes.items():
    for img in v:
        if img not in image_index:
            image_index[img]=x
            x+=1

In [46]:
user_list=[]
image_list=[]
for line in appreciate_data:
    line = line.strip()
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])
    user_list.append(user_index[user_id])
    image_list.append(image_index[item_id])

In [47]:
from scipy.sparse import csr_matrix
row = np.array(user_list)
col = np.array(image_list)
data = np.ones(row.shape[0])
csr_ob=csr_matrix((data, (row, col)), shape=(len(user_index), len(image_index)))

In [48]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000)
matrix=svd.fit_transform(csr_ob)
# print(svd.explained_variance_ratio_.sum())

In [49]:
!apt install libomp-dev
!pip install faiss
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [50]:
d = len(matrix[0]) 
mat = np.asarray(matrix).astype('float32')
index = faiss.IndexFlatL2(d)
index.add(mat)

In [51]:
D, I = index.search(mat, int(k_value)*2)
# print(I)

In [52]:
usr_row_num=user_index[user]
listt=[]
k=int(k_value)
for i in range(1,k*2):
    listt.append(index_user[I[usr_row_num][i]])
# print(listt)

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [53]:
images={}
def neighbors(user,k_value):
    """ returns an iterable object (like list or generator) """
    for id in listt:
        images[id]=list()
        for item in user_likes[id]:
            if item not in user_likes[user]:
                images[id].append(item)
        if(len(images[id])!=0):
            k_value=k_value-1
        if k_value==0:
            return images
    pass

### Answer the following questions:

#### Q1. **Explain how your choice of library works**

Faiss (Facebook AI Similarity Search) provides several similarity search methods that span a wide spectrum of usage trade-offs.It gives a trade-off between speed and accuracy for a fixed memory usage by focusing on methods that compress the original vectors, because they’re the only ones that scale to data sets of billions of vectors. The algorithm processes the 1 billion vectors and encodes them into an index to reduce index memory size and increase search speed.

#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

FAISS makes use of both Euclidean distance and dot product for comparing vectors. It uses  GPU, so search can process calculations in parallel rather than in series and also performs Principle Component Analysis for dimensionality reduction. It divides data points into areas and finds centroid areas, and performs an exhaustive search only on the data points having the closest centroid to the test data point, we can get a complexity of O(sqrt(n)) for calculation of distances and the top k could be found out in O(sqrt(n) + k*log(n)).
Faiss on an average performs 300 times faster than the vanilla KNN.  Finding k nearest neighbors for one data point takes O(nd) time in vanilla KNN.

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

ScaNN	
ScaNN (Scalable Nearest Neighbors) is a method for efficient vector similarity search at a large scale and shows a strong performance with large datasets. It includes search space pruning and quantization for maximum inner product search and it also supports other distance functions such as Euclidean distance. It can be configured to fit datasets with different sizes. It has both TensorFlow and Python APIs.


Faiss is a library for efficient similarity search and clustering of dense vectors. It assumes that the instances are represented as vectors and are identified by an integer, and that the vectors can be compared with L2 (Euclidean) distances or dot products. Vectors that are similar to a query vector are those that have the lowest L2 distance or the highest dot product with the query vector. It also supports cosine similarity, since this is a dot product on normalized vectors. 


Annoy (Approximate Nearest Neighbors Oh Yeah) is a C++ library with Python bindings to search for points in space that are close to a given query point.
For example, if we 10,000,000, i.e. 10M entities. In KNN, we will need to find cosine similarity and sort (10M * 10M) pairs and the complexity grows to O(n^2).
In annoy, we dont need to calculate and store the similarities and distances of each point with other. It considers only the space with the cluster of points and does not care about the points that have a zero chace of appearing in the neighbourhood. Thus, requires us to build a cluster index first , which basically decides the X leaders with which you'll find similarities. Then, the cluster index is used to find Approximate KNN for all given points. This reduces the complexity from O(n2) to just O(n*k). Storage becomes easier, Calculation becomes faster.

### Open the output file to write all the lines to the file

In [54]:
outFile = open('output.txt', 'w')
images=neighbors(user,k_value)
for n_user,lis in images.items():
    for item_id in lis:
        outFile.write(str(item_id) + ' ' + str(n_user) + '\n')

outFile.close()